In [1]:
START=0;
END=100;

### Goal: Try again on estimation on region
Based on all the features we get this time.

In [2]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.preprocessing import MinMaxScaler;
from sklearn.metrics import mean_squared_error;

import datetime;

2.6.0


In [3]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [4]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [5]:
MONTHS = 60;
SPLIT = 48; # 2015-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 24;
WINDOW_SIZE = 3;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [6]:
multi_data = pd.read_csv('../data/full_dataset_unscaled.csv', index_col=0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
                 axis = 1, inplace = True);

multi_data.head()

,zri,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,percent_rental_units_vacant,percent_rental_units_occupied,percent_graduate_deg,percent_bachelors,percent_associates,percent_highschool,percent_less_highschool,percent_commute_public_transport,percent_commute_less_30,percent_buildings_less_10_units,percent_buildings_10_19_units,percent_buildings_20_49_units,percent_buildings_50+_units,percent_commute_30_to_59,percent_commute_60_to_89,percent_commute_90_more,percent_new_city,percent_new_unit,percent_units_owner_occupied,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing,gun range,gun control,gun violence,job opportunities,unemployment,retirement,layoff,lgbt,same sex marriage,they,pronouns,black lives matter,political correctness,make america great again,euthanasia,getaway,places to go,flight tickets,twitter,hashtag,fake news,hurricane,wildfire,flood,fire,trader joe's,whole foods,lululemon,thrift,condos for rent,duplex apartments for rent,townhomes for rent,townhouses for rent,home for rent,house for rent,townhome for rent,townhouse for rent,apartment for rent,studio for rent,1 bedroom for rent,3 bedroom for rent,starbucks,total_firms,job_creation_rate,job_destruction_rate,startup_firms,state_local_perc
zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1013,946.0,0.714892,0.023829,0.013748,0.230129,0.000113,0.004539,0.224062,0.314378,0.316595,0.144965,0.057735,0.942265,0.029555,0.062456,0.257328,0.383459,0.218492,0.022609,0.731245,0.883422,0.030202,0.036863,0.049514,0.226387,0.021712,0.016727,0.109462,0.040593,0.423344,62.333333,20935.75,0.199214,21846.25,0.105077,0.014618,36.508333,0.502083,0.417325,0.057131,0.0,0.0,0,-1.569568,-0.986047,-0.723220,0.096863,3.466751,-1.425905,-0.823717,-0.141562,-0.588436,-1.395901,-1.326593,-0.509999,0.0,-0.514772,-0.150792,-0.468627,-1.304734,-1.378408,1.004233,0.448996,-0.659443,-0.285509,-0.443099,0.944722,1.162014,-0.662346,0.291193,-0.212701,-0.377642,0.363614,0.0,0.0,0.0,-0.629886,-0.622713,0.0,0.0,-1.390248,-1.181638,-0.872013,-1.100628,-1.712329,7340.000000,10.221000,8.228000,383.0,0.176793
1013,950.0,0.715367,0.024195,0.014023,0.229024,0.000226,0.004587,0.223443,0.316319,0.315346,0.144892,0.057503,0.942497,0.029598,0.062682,0.255708,0.385186,0.217890,0.023198,0.733361,0.883598,0.030477,0.036368,0.049557,0.224662,0.022151,0.016350,0.107248,0.040401,0.421607,62.666667,20931.50,0.196885,21870.50,0.105835,0.014466,36.416667,0.502517,0.416750,0.057072,0.0,0.0,0,0.795072,0.879861,0.254251,1.061035,1.934232,-1.217118,-0.823717,-1.071717,-0.588436,-1.473019,-1.326593,-0.509999,0.0,-0.514772,-0.150792,0.019798,-0.015282,-0.847070,1.303017,0.448996,-0.659443,-0.431227,-0.443099,-1.199445,0.225152,0.435704,0.719432,0.157345,-0.826044,-0.142702,0.0,0.0,0.0,-1.612318,-0.622713,0.0,0.0,-0.050671,0.701568,0.097872,-1.100628,-1.241616,7335.583333,10.078917,8.328083,382.0,0.176313
1013,958.0,0.715842,0.024561,0.014299,0.227919,0.000339,0.004634,0.222823,0.318260,0.314097,0.144820,0.057270,0.942730,0.029641,0.062908,0.254088,0.386912,0.217289,0.023787,0.735477,0.883775,0.030752,0.035873,0.049601,0.222936,0.022591,0.015974,0.105034,0.040209,0.419870,63.000000,20927.25,0.194555,21894.75,0.106592,0.014314,36.325000,0.502950,0.416175,0.057013,0.0,0.0,0,0.795072,0.879861,-0.723220,-0.385223,0.991144,0.348785,-0.823717,-1.536795,-0.086484,0.069350,0.732224,-0.509999,0.0,-0.514772,1.382217,-0.794244,-1.304734,-0.847070,1.751194,0.448996,0.570434,-0.285509,-0.443099,-0.127361,1.295851,-0.662346,0.230016,-0.952793,-0.527109,-0.902177,0.0,0.0,0.0,0.025069,0.565707,0.0,0.0,-0.172450,-0.475436,-0.872013,-1.100628,-1.268533,7331.166667,9.936833,8.428167,381.0,0.175832
1013,958.0,0.716318,0.024927,0.014

In [7]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

# multi_data = pd.read_csv('gdrive/My Drive/full_dataset_unscaled.csv', index_col=0);
# zip_ids = multi_data.index.unique();

# multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
#                  axis = 1, inplace = True);

# multi_data.head()

In [8]:
FEATURES = multi_data.shape[1] - 1;

feature_name = list(multi_data.columns);

In [10]:
# In our first try, just look at the zip codes in NY. zip:10001-14905
multi_zip = zip_ids;
# list(multi_data[(multi_data.index >= 10001) & (multi_data.index <= 14905)].index.unique());
print(len(multi_zip))

2


### Utility functions

In [11]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1], window[-1][0]));
    dataset = dataset.batch(batch_size);
    return dataset;

In [12]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [13]:
def NN_forecast(model, series_transformed, pure=True):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, :]))

    results = [float(x[-1][0]) for x in forecast];
    actual = list(series_transformed[WINDOW_SIZE:, 0]);
    
    if not pure:
        return results, actual;

    timeline = range(WINDOW_SIZE, MONTHS);
    time_test = range(SPLIT, MONTHS);
    forecast = series_transformed[SPLIT - WINDOW_SIZE:,:].copy();

    for time in range(TEST_LENGTH): # Change temp[time + WINDOW_SIZE]
        forecast[time + WINDOW_SIZE, 0] =\
        model.predict(forecast[np.newaxis, time:time + WINDOW_SIZE, :])[-1][0];

    pure_forecast = forecast[WINDOW_SIZE:,0];
    
    print(len(results), len(actual), len(pure_forecast))
    
    return results, actual, pure_forecast;

### Load model

In [14]:
model = tf.keras.models.load_model('./saved_models/ZRI_and_all.h5');

In [15]:
collection_importance = [];

for zip_num in multi_zip:
    scaler = MinMaxScaler();
    series = np.array(multi_data[multi_data.index == zip_num]);
    series_transformed = scaler.fit_transform(series);

    # Forecasting
    results, actual = NN_forecast(model, series_transformed, pure=False);

    # Compute MSE
    mse = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5;
    # Usually we need to multiply by scaler.data_range_[0], but can ignore this.
    TRIALS = 1;
    PLOT_IMPORTANCE = True;
    
    perm_importance = [zip_num];
    for i in range(len(feature_name)):
        mse_feat = [];
        for _ in range(TRIALS):
            seq_perm = series_transformed.copy();
            seq_perm[:, i] = np.random.permutation(seq_perm[:,i])
            results, actual = NN_forecast(model, seq_perm, pure = False);
            mse_feat.append(mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5);

        perm_importance.append(sum(mse_feat)/TRIALS/mse - 1.0);
        # print("{}, {:.4f}".format(feature_name[i], sum(mse_feat)/TRIALS/mse - 1.0));

    collection_importance.append(perm_importance);


zri, 11.2082
percent_white, -0.0081
percent_black, 0.0041
percent_asian, 0.0008
percent_hispanic, -0.0144
percent_native_am, -0.0023
percent_other_race, -0.0167
percent_0_17, 0.0159
percent_18_39, 0.0284
percent_40_64, 0.0147
percent_65+, -0.0068
percent_rental_units_vacant, 0.4230
percent_rental_units_occupied, 0.1192
percent_graduate_deg, -0.0158
percent_bachelors, 0.0079
percent_associates, 0.0048
percent_highschool, 0.0034
percent_less_highschool, 0.0021
percent_commute_public_transport, 0.0233
percent_commute_less_30, 0.0067
percent_buildings_less_10_units, 0.0142
percent_buildings_10_19_units, 0.0184
percent_buildings_20_49_units, -0.0034
percent_buildings_50+_units, 0.0182
percent_commute_30_to_59, 0.0286
percent_commute_60_to_89, -0.0138
percent_commute_90_more, 0.0036
percent_new_city, 0.0037
percent_new_unit, -0.0030
percent_units_owner_occupied, 0.0148
median_building_age, -0.0040
income_per_capita, -0.0027
poverty_rate, -0.0049
total_pop, 0.0008
percent_workforce_unemployed

In [16]:
importance_df = pd.DataFrame(collection_importance, columns = ["zip"] + feature_name);
importance_df.to_csv("./Results in CSV files/Importance.csv", index = False);

In [17]:
importance_df

,zip,zri,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,percent_rental_units_vacant,percent_rental_units_occupied,percent_graduate_deg,percent_bachelors,percent_associates,percent_highschool,percent_less_highschool,percent_commute_public_transport,percent_commute_less_30,percent_buildings_less_10_units,percent_buildings_10_19_units,percent_buildings_20_49_units,percent_buildings_50+_units,percent_commute_30_to_59,percent_commute_60_to_89,percent_commute_90_more,percent_new_city,percent_new_unit,percent_units_owner_occupied,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing,gun range,gun control,gun violence,job opportunities,unemployment,retirement,layoff,lgbt,same sex marriage,they,pronouns,black lives matter,political correctness,make america great again,euthanasia,getaway,places to go,flight tickets,twitter,hashtag,fake news,hurricane,wildfire,flood,fire,trader joe's,whole foods,lululemon,thrift,condos for rent,duplex apartments for rent,townhomes for rent,townhouses for rent,home for rent,house for rent,townhome for rent,townhouse for rent,apartment for rent,studio for rent,1 bedroom for rent,3 bedroom for rent,starbucks,total_firms,job_creation_rate,job_destruction_rate,startup_firms,state_local_perc
0,1013,11.208250,-0.008097,0.004079,0.000809,-0.014398,-0.002254,-0.016673,0.015850,0.028439,0.014656,-0.006846,0.423026,0.119190,-0.015834,0.007943,0.004778,0.003418,0.002134,0.023297,0.006654,0.014238,0.018374,-0.003433,0.018233,0.028647,-0.013824,0.003625,0.003713,-0.002957,0.014802,-0.004021,-0.002656,-0.004857,0.000784,-0.019356,0.022674,-0.002779,-0.003982,0.011772,-0.00245,0.0,0.0,0.0,0.002037,0.001628,-0.008776,0.003100,0.008289,-0.013365,-0.006194,-0.001084,0.000941,0.000480,-0.019494,-0.042324,0.0,-0.005543,-0.000974,-0.018141,0.018844,0.008184,0.007458,0.000087,0.010422,-0.010337,-0.022862,-0.012425,0.005112,-0.000608,0.005896,-0.002026,0.001003,-0.001108,0.0,0.0,0.0,0.006120,-0.000231,0.0,0.0,-0.003824,0.006231,0.005761,0.004097,-0.001828,0.004264,-0.010764,0.012865,0.014062,0.017564
1,1020,14.024062,-0.017325,0.021001,0.002194,0.005115,-0.037027,-0.027061,0.112718,0.033426,0.013771,-0.004559,0.777680,0.674554,0.001520,0.001219,0.002058,0.024875,0.011337,0.002099,0.068572,0.085879,0.010964,0.018349,0.006562,0.068352,0.010378,0.028190,0.012281,0.005505,0.010646,0.006406,0.015022,0.001128,-0.017518,0.065423,0.061774,-0.008092,0.002567,-0.008729,0.02153,0.0,0.0,0.0,-0.000056,-0.022524,0.000880,0.010652,-0.008386,-0.009223,-0.009640,-0.001058,0.000895,0.005415,-0.007516,-0.033329,0.0,-0.041786,0.007412,-0.040997,0.012896,-0.003350,0.021324,0.015608,0.012054,-0.015909,-0.030307,0.003964,0.007864,0.010097,-0.006087,-0.002878,-0.002793,-0.003753,0.0,0.0,0.0,0.003085,-0.004783,0.0,0.0,0.006701,-0.005653,0.000080,-0.015322,-0.001944,-0.010480,0.016175,0.020827,0.037532,0.065893
